In [269]:
import pandas as pd
import numpy as np
import random
import datetime

### dimFactories

In [270]:
factories = pd.DataFrame(pd.read_csv('./done_csv/dimOrder.csv')['factory_code'].unique()).rename({0: 'factory_code'}, axis=1)

In [271]:
factories['factory_id'] = factories.index + 1
factories['factory_name'] = pd.Series([ "Vinnytsia", "Buffalo", "Bila Tserkva", "Deva", "Weimar", "Znojmo",
    "Royan", "Montevideo", "Clifton", "Daugavpils", "Skadovsk", "Vajjak", "Norwich", "Tesinets", "Espoo",
    "Arpad", "Thessaloniki", "Blankenburg", "Dnipro", "Utsen", "Skovorodynivka", "Dagestan", "Savastopol"
])

In [272]:
factories = factories.iloc[:, [1, 2, 0]]

In [273]:
factories.to_csv('./done_csv/dimFactory.csv', index=None)

### dimParcel

In [274]:
orders = pd.read_csv('./done_csv/dimOrder.csv')
products = pd.read_csv('./done_csv/dimProduct.csv')

In [275]:
orders['order_date'] = orders['order_date'].apply(pd.to_datetime).dt.date

C:\Users\Ivan\AppData\Local\Temp\ipykernel_16844\1397814682.py:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  orders['order_date'] = orders['order_date'].apply(pd.to_datetime).dt.date


In [276]:
order_sample = orders.sort_values('order_date', ascending=False).head(100)

In [277]:
def get_qt_per_parcil(product_name):
    if product_name == None : return None
    return products.loc[products['product_name'] == product_name].iloc[0].loc['qt_per_parcel']

In [278]:
order_sample['qt_per_parcel'] = order_sample['product_name'].apply(get_qt_per_parcil)

In [279]:
order_sample['order_qt'] = order_sample['order_qt'].astype(int)
order_sample['qt_per_parcel'] = order_sample['qt_per_parcel'].astype(float)

In [280]:
order_sample['parcels_qt_full'] = pd.Series(np.ceil(order_sample['order_qt'] / order_sample['qt_per_parcel']))

In [281]:
order_sample = order_sample.loc[~order_sample['parcels_qt_full'].isna()].head(10)

In [282]:
parcels = pd.DataFrame(columns=['parcel_id', 'fk_order_id','№ in order', 'parcel_status', 'parcel_created_date'])

In [283]:
counter = 0
for order_code in order_sample['order_code']:
    n_parcels = int(order_sample.loc[order_sample['order_code'] == order_code]['parcels_qt_full'].values[0])
    order_date = order_sample.loc[order_sample['order_code'] == order_code]['order_date'].values[0]
    last_date = order_date
    for n in range(n_parcels):
        parcels.loc[counter]= [counter+1, order_code, n+1, 1, last_date]
        last_date += datetime.timedelta(days=random.choices([0, 1, 2, 3], [.5, .3, .15, .05])[0])
        counter += 1

In [284]:
parcels.to_csv('./done_csv/dimParcels.csv', index=None)

In [285]:
parcels.to_json('./done_json/dimParcels.json', orient='records')

### factParcelRun

In [366]:
parcel_runs = pd.DataFrame(columns=['fk_parcel_id', 'varnish_type', 'run_number', 'varnish_amount_kg', 'solvent_amount_kg', 'run_status', 'run_date'])

In [384]:
def get_order_details(order):
    return order_sample.loc[order_sample['order_code'] == order].iloc[0]

def get_product_details(product_name):
    return products.loc[products['product_name'] == product_name].iloc[0]

def get_parcel_details(parcel_id):
    return parcels.loc[parcels['parcel_id'] == parcel_id].iloc[0]

def get_run_data(parcel_id):
    return parcel_runs.loc[parcel_runs['fk_parcel_id'] == parcel_id]

def get_varnishings(parcel_id):
    return get_product_details(get_order_details(get_parcel_details(parcel_id).iloc[1]).iloc[2]).loc[['varnishing_1', 'varnishing_2', 'varnishing_3', 'varnishing_4']]

In [378]:
counter = 0
for parcel in parcels['parcel_id']:
    varnishings = get_varnishings(int(parcel)).dropna()
    parcel_date = get_parcel_details(17).iloc[4]
    last_date = parcel_date
    for i in range(len(varnishings)):
        parcel_runs.loc[counter] = [parcel, varnishings[i], i+1, random.randint(10, 1500), random.randint(1, 500), 1, last_date]
        last_date += datetime.timedelta(days=random.choices([1, 2, 3, 4], [.5, .3, .15, .05])[0])
        counter += 1

C:\Users\Ivan\AppData\Local\Temp\ipykernel_16844\1138553804.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  parcel_runs.loc[counter] = [parcel, varnishings[i], i+1, random.randint(10, 1500), random.randint(1, 500), 1, last_date]
C:\Users\Ivan\AppData\Local\Temp\ipykernel_16844\1138553804.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-12-01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  parcel_runs.loc[counter] = [parcel, varnishings[i], i+1, random.randint(10, 1500), random.randint(1, 500), 1, last_date]
C:\Users\Ivan\AppData\Local\Temp\ipykernel_16844\1138553804.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version,

In [385]:
parcel_runs

,fk_parcel_id,varnish_type,run_number,varnish_amount_kg,solvent_amount_kg,run_status,run_date
0,17,PL1019-34,1,89,182,1,2024-12-01
1,17,816361,2,1402,399,1,2024-12-02
2,18,PL1019-34,1,347,82,1,2024-12-01
3,18,816361,2,199,305,1,2024-12-03
4,19,816314,1,1046,444,1,2024-12-01
...,...,...,...,...,...,...,...
131,104,PL1014-59,2,1275,450,1,2024-12-03
132,105,28S30WB-1,1,926,100,1,2024-12-01
133,105,PL1014-59,2,586,242,1,2024-12-03
134,106,28S30WB-1,1,1001,249,1,2024-12-01


In [388]:
get_run_data(104)

,fk_parcel_id,varnish_type,run_number,varnish_amount_kg,solvent_amount_kg,run_status,run_date
130,104,28S30WB-1,1,764,67,1,2024-12-01
131,104,PL1014-59,2,1275,450,1,2024-12-03


In [389]:
parcel_runs.to_csv('./done_csv/factParcelRuns.csv', index=None)